## CNN

In [365]:
import os
import numpy as np
import pandas as pd
from PIL import Image
from tqdm import tqdm
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.metrics import accuracy_score, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical


In [366]:
img_size = (64, 64)  # Размер изображения
data_dir = "train"
csv_path = "train.csv"
df = pd.read_csv(csv_path)


In [367]:
# Преобразование меток классов в числовой формат
label_encoder = LabelEncoder()
df['Class'] = label_encoder.fit_transform(df['Class'])
num_classes = len(df['Class'].unique())

In [368]:
def load_and_preprocess_image(image_path):
    try:
        img = Image.open(image_path).convert('RGB')
        img = img.resize(img_size)
        img = np.array(img) / 255.0  # Нормализация
        return img
    except Exception as e:
        print(f"Ошибка при загрузке {image_path}: {e}")
        return None

In [369]:
X = []
y = []
for _, row in tqdm(df.iterrows(), total=len(df)):
    image_id, label = row['ID'], row['Class']
    image_path = os.path.join(data_dir, image_id)
    img = load_and_preprocess_image(image_path)
    if img is not None:
        X.append(img)
        y.append(label)

X = np.array(X)
y = np.array(y)

  0%|          | 0/19906 [00:00<?, ?it/s]

100%|██████████| 19906/19906 [00:11<00:00, 1663.05it/s]


In [370]:
y = to_categorical(y, num_classes=num_classes)

In [371]:
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42)
X_valid, X_test, y_valid, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

print(f"Форма X_train: {X_train.shape}, y_train: {y_train.shape}")
print(f"Форма X_valid: {X_valid.shape}, y_valid: {y_valid.shape}")
print(f"Форма X_test: {X_test.shape}, y_test: {y_test.shape}")


Форма X_train: (15924, 64, 64, 3), y_train: (15924, 3)
Форма X_valid: (1991, 64, 64, 3), y_valid: (1991, 3)
Форма X_test: (1991, 64, 64, 3), y_test: (1991, 3)


In [372]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')  
])

c:\Users\nosko\OneDrive\Рабочий стол\cse2\venv\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [373]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

history = model.fit(X_train, y_train,
                    epochs=10,  
                    batch_size=32,
                    validation_data=(X_valid, y_valid))

test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Точность на тестовой выборке: {test_accuracy:.2f}")


Epoch 1/10
498/498 ━━━━━━━━━━━━━━━━━━━━ 10s 19ms/step - accuracy: 0.5677 - loss: 0.9193 - val_accuracy: 0.6645 - val_loss: 0.7676
Epoch 2/10
498/498 ━━━━━━━━━━━━━━━━━━━━ 9s 18ms/step - accuracy: 0.6636 - loss: 0.7813 - val_accuracy: 0.6580 - val_loss: 0.7190
Epoch 3/10
498/498 ━━━━━━━━━━━━━━━━━━━━ 9s 19ms/step - accuracy: 0.6982 - loss: 0.7141 - val_accuracy: 0.6851 - val_loss: 0.6844
Epoch 4/10
498/498 ━━━━━━━━━━━━━━━━━━━━ 9s 18ms/step - accuracy: 0.7215 - loss: 0.6606 - val_accuracy: 0.7283 - val_loss: 0.6239
Epoch 5/10
498/498 ━━━━━━━━━━━━━━━━━━━━ 9s 18ms/step - accuracy: 0.7377 - loss: 0.6206 - val_accuracy: 0.7378 - val_loss: 0.6200
Epoch 6/10
498/498 ━━━━━━━━━━━━━━━━━━━━ 9s 18ms/step - accuracy: 0.7539 - loss: 0.5778 - val_accuracy: 0.7594 - val_loss: 0.5931
Epoch 7/10
498/498 ━━━━━━━━━━━━━━━━━━━━ 9s 18ms/step - accuracy: 0.7684 - loss: 0.5439 - val_accuracy: 0.7564 - val_loss: 0.5986
Epoch 8/10
498/498 ━━━━━━━━━━━━━━━━━━━━ 9s 18ms/step - accuracy: 0.7850 - loss: 0.5140 - val_acc

In [374]:
model.save("cnn_model.h5")

In [375]:
sample_idx = 0
sample_image = X_test[sample_idx]
sample_label = np.argmax(y_test[sample_idx])
prediction = np.argmax(model.predict(np.expand_dims(sample_image, axis=0)))

print(f"Реальный класс: {sample_label}, Предсказанный класс: {prediction}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
Реальный класс: 0, Предсказанный класс: 0


In [376]:
y_train_pred = np.argmax(model.predict(X_train), axis=1)
y_valid_pred = np.argmax(model.predict(X_valid), axis=1)
y_test_pred = np.argmax(model.predict(X_test), axis=1)


y_train_true = np.argmax(y_train, axis=1)
y_valid_true = np.argmax(y_valid, axis=1)
y_test_true = np.argmax(y_test, axis=1)


train_mseCNN = mean_squared_error(y_train_true, y_train_pred)
train_r2CNN = r2_score(y_train_true, y_train_pred)

valid_mse = mean_squared_error(y_valid_true, y_valid_pred)
valid_r2 = r2_score(y_valid_true, y_valid_pred)

test_mse = mean_squared_error(y_test_true, y_test_pred)
test_r2 = r2_score(y_test_true, y_test_pred)

accuracyCNN = accuracy_score(y_test_true, y_test_pred)

print(f"Точность моделиCNN: {accuracyCNN:.2f}")
print(f"Train MSECNN: {train_mseCNN:.4f}, Train R^2CNN: {train_r2CNN:.4f}")



498/498 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
Точность моделиCNN: 0.77
Train MSECNN: 0.3579, Train R^2CNN: 0.5727


## _____________________________________________________________________________________________________________

## Random Forest

In [377]:
csv_path = "train.csv"
data_dir = "train"
df = pd.read_csv(csv_path)

In [378]:
label_encoder = LabelEncoder()
df['Class'] = label_encoder.fit_transform(df['Class'])

In [379]:
def extract_features(image_path):
    try:
        image = Image.open(image_path).convert('RGB')
        image = image.resize((64, 64))
        features = np.array(image).flatten()
        return features
    except Exception as e:
        print(f"Ошибка при обработке {image_path}: {e}")
        return None

In [380]:
X = []
y = []
for _, row in tqdm(df.iterrows(), total=len(df)):
    image_id, label = row['ID'], row['Class']
    image_path = os.path.join(data_dir, image_id)
    features = extract_features(image_path)
    if features is not None and len(features) == 12288:
        X.append(features)
        y.append(label)

X = np.array(X)
y = np.array(y)

print(f"Общее количество данных: {len(X)}")
print(f"Форма массива признаков: {X.shape}")
print(f"Форма меток: {y.shape}")

100%|██████████| 19906/19906 [00:10<00:00, 1824.31it/s]


Общее количество данных: 19906
Форма массива признаков: (19906, 12288)
Форма меток: (19906,)


In [381]:
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42)
X_valid, X_test, y_valid, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

print(f"Форма обучающей выборки: {X_train.shape}")
print(f"Форма валидационной выборки: {X_valid.shape}")
print(f"Форма тестовой выборки: {X_test.shape}")

Форма обучающей выборки: (15924, 12288)
Форма валидационной выборки: (1991, 12288)
Форма тестовой выборки: (1991, 12288)


In [382]:
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [383]:
y_train_pred = model.predict(X_train)
y_valid_pred = model.predict(X_valid)
y_test_pred = model.predict(X_test)

In [384]:
train_mseRF = mean_squared_error(y_train, y_train_pred)
train_r2RF = r2_score(y_train, y_train_pred)

valid_mse = mean_squared_error(y_valid, y_valid_pred)
valid_r2 = r2_score(y_valid, y_valid_pred)

test_mse = mean_squared_error(y_test, y_test_pred)
test_r2 = r2_score(y_test, y_test_pred)

accuracyRF = accuracy_score(y_test, y_test_pred)
print(f"Точность моделиRF: {accuracyRF:.2f}")
print(f"Train MSERF: {train_mseRF:.4f}, Train R^2RF: {train_r2RF:.4f}")


Точность моделиRF: 0.70
Train MSERF: 0.0003, Train R^2RF: 0.9996


## _______________________________________________________________________________________________________________________________

## Линейная регрессия (LR)

In [385]:
image_size = (64, 64)  

csv_path = "train.csv"
data = pd.read_csv(csv_path)

In [386]:
def load_images_and_labels(image_folder, data, image_size):
    images = []
    labels = []
    for _, row in tqdm(data.iterrows(), total=data.shape[0], desc="Загрузка изображений"):
        img_path = os.path.join(image_folder, row["ID"])
        label = row["Class"]
        try:
            img = Image.open(img_path).convert("L") 
            img = img.resize(image_size)
            images.append(np.array(img).flatten())
            labels.append(label)
        except Exception as e:
            print(f"Ошибка при обработке файла {img_path}: {e}")
    return np.array(images), np.array(labels)

In [387]:
image_folder = "train"


In [388]:
X, y = load_images_and_labels(image_folder, data, image_size)

Загрузка изображений: 100%|██████████| 19906/19906 [00:09<00:00, 2085.51it/s]


In [389]:
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

In [390]:
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42)
X_valid, X_test, y_valid, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)



In [391]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)


In [392]:
model = LinearRegression()
model.fit(X_train, y_train)


LinearRegression()

In [393]:
y_train_pred = model.predict(X_train)
y_valid_pred = model.predict(X_valid)
y_test_pred = model.predict(X_test)

In [394]:
train_mseLR = mean_squared_error(y_train, y_train_pred)
train_r2LR = r2_score(y_train, y_train_pred)

train_accuracyLR = accuracy_score(y_train, np.round(y_train_pred))

# Вывод метрик
print(f"Точность моделиLR: {train_accuracyLR:.2f}")
print(f"MSELR: {train_mseLR:.4f},R^2LR: {train_r2LR:.4f}")



Точность моделиLR: 0.44
MSELR: 0.5230,R^2LR: 0.3755


## _________________________________________________________________________________________________________

## Вывод

In [395]:
data = {
    "Модель": ["Нейронная сеть (СNN)", "Случайный лес (RF)", "Линейная регрессия (LR)"],
    "Точность": [accuracyCNN, accuracyRF, train_accuracyLR],
    "MSE": [train_mseCNN, train_mseRF, train_mseLR],
    "R²": [train_r2CNN, train_r2RF, train_r2LR]
    }


df = pd.DataFrame(data)

print("\nСравнение моделей:")
print(df)


Сравнение моделей:
                    Модель  Точность       MSE        R²
0     Нейронная сеть (СNN)  0.766951  0.357887  0.572690
1       Случайный лес (RF)  0.701657  0.000314  0.999625
2  Линейная регрессия (LR)  0.438646  0.523001  0.375548
